In [22]:
import numpy as np
import pandas as pd
import os
import sqlite3
import matplotlib.pyplot as plt
import pathlib as path  # For relative path

In [23]:
# Load the csv FastFoodNutritionMenuV2 from the resources folder. 
csv_path = ('../resources/FastFoodNutritionMenuV2.csv')
fastfooddata = pd.read_csv(csv_path)

# display the first 5 rows of the data
fastfooddata.head()


,Company,Item,Calories,Calories from\nFat,Total Fat\n(g),Saturated Fat\n(g),Trans Fat\n(g),Cholesterol\n(mg),Sodium \n(mg),Carbs\n(g),Fiber\n(g),Sugars\n(g),Protein\n(g),Weight Watchers\nPnts
0,McDonald’s,Hamburger,250,80,9,3.5,0.5,25,520,31,2,6,12,247.5
1,McDonald’s,Cheeseburger,300,110,12,6,0.5,40,750,33,2,6,15,297
2,McDonald’s,Double Cheeseburger,440,210,23,11,1.5,80,1150,34,2,7,25,433
3,McDonald’s,McDouble,390,170,19,8,1,65,920,33,2,7,22,383
4,McDonald’s,Quarter Pounder® with Cheese,510,230,26,12,1.5,90,1190,40,3,9,29,502


In [24]:
# Cleaning the column names: 
fastfooddata.columns = fastfooddata.columns.str.replace('\n', ' ').str.strip()

# Display the columns
fastfooddata.head()

,Company,Item,Calories,Calories from Fat,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Carbs (g),Fiber (g),Sugars (g),Protein (g),Weight Watchers Pnts
0,McDonald’s,Hamburger,250,80,9,3.5,0.5,25,520,31,2,6,12,247.5
1,McDonald’s,Cheeseburger,300,110,12,6,0.5,40,750,33,2,6,15,297
2,McDonald’s,Double Cheeseburger,440,210,23,11,1.5,80,1150,34,2,7,25,433
3,McDonald’s,McDouble,390,170,19,8,1,65,920,33,2,7,22,383
4,McDonald’s,Quarter Pounder® with Cheese,510,230,26,12,1.5,90,1190,40,3,9,29,502


In [25]:
# Debug: Print the number of entries for each company
print(fastfooddata['Company'].value_counts())

Company
McDonald’s     329
KFC            218
Burger King    190
Taco Bell      183
Wendy’s        154
Pizza Hut       74
Name: count, dtype: int64


In [26]:
# Convert relevant columns to numeric, forcing errors to NaN
fastfooddata['Sugars (g)'] = pd.to_numeric(fastfooddata['Sugars (g)'], errors='coerce')
fastfooddata['Calories'] = pd.to_numeric(fastfooddata['Calories'], errors='coerce')

In [27]:
# Debug: Print the number of entries for each company
print(fastfooddata['Company'].value_counts())

Company
McDonald’s     329
KFC            218
Burger King    190
Taco Bell      183
Wendy’s        154
Pizza Hut       74
Name: count, dtype: int64


In [28]:
# Check for missing values in the dataset
missing_values = fastfooddata.isnull().sum()
print("\nMissing values in each column:\n", missing_values)

# Display the first few rows of the dataset to understand its structure
print("\nFirst few rows of the dataset:\n", fastfooddata.head())


Missing values in each column:
 Company                   0
Item                      0
Calories                 15
Calories from Fat       506
Total Fat (g)            57
Saturated Fat (g)        57
Trans Fat (g)            57
Cholesterol (mg)          1
Sodium  (mg)              1
Carbs (g)                57
Fiber (g)                57
Sugars (g)               30
Protein (g)              57
Weight Watchers Pnts    261
dtype: int64

First few rows of the dataset:
       Company                           Item  Calories Calories from Fat  \
0  McDonald’s                      Hamburger     250.0                80   
1  McDonald’s                   Cheeseburger     300.0               110   
2  McDonald’s            Double Cheeseburger     440.0               210   
3  McDonald’s                       McDouble     390.0               170   
4  McDonald’s   Quarter Pounder® with Cheese     510.0               230   

  Total Fat (g) Saturated Fat (g) Trans Fat (g) Cholesterol (mg) Sodium 

In [29]:
# list columns in the dataset
print("\nColumns in the dataset:\n", fastfooddata.columns)


Columns in the dataset:
 Index(['Company', 'Item', 'Calories', 'Calories from Fat', 'Total Fat (g)',
       'Saturated Fat (g)', 'Trans Fat (g)', 'Cholesterol (mg)',
       'Sodium  (mg)', 'Carbs (g)', 'Fiber (g)', 'Sugars (g)', 'Protein (g)',
       'Weight Watchers Pnts'],
      dtype='object')


In [30]:
# Cleaning the data. We have a lot of missing data from Calories from Fat and Weight watchers points.
cleaned_df = fastfooddata.drop(columns=['Calories from Fat', 'Weight Watchers Pnts'])
print("\nFirst few rows of the cleaned dataset:\n", cleaned_df.head())


First few rows of the cleaned dataset:
       Company                           Item  Calories Total Fat (g)  \
0  McDonald’s                      Hamburger     250.0             9   
1  McDonald’s                   Cheeseburger     300.0            12   
2  McDonald’s            Double Cheeseburger     440.0            23   
3  McDonald’s                       McDouble     390.0            19   
4  McDonald’s   Quarter Pounder® with Cheese     510.0            26   

  Saturated Fat (g) Trans Fat (g) Cholesterol (mg) Sodium  (mg) Carbs (g)  \
0               3.5           0.5               25          520        31   
1                 6           0.5               40          750        33   
2                11           1.5               80         1150        34   
3                 8             1               65          920        33   
4                12           1.5               90         1190        40   

  Fiber (g)  Sugars (g) Protein (g)  
0         2         6.0  

In [31]:
# Debug: Print the number of entries for each company
print(cleaned_df['Company'].value_counts())

Company
McDonald’s     329
KFC            218
Burger King    190
Taco Bell      183
Wendy’s        154
Pizza Hut       74
Name: count, dtype: int64


In [32]:
# Check the new missing values in the cleaned dataset
missing_values_cleaned = cleaned_df.isnull().sum()
print("\nMissing values in each column of the cleaned dataset:\n", missing_values_cleaned)


Missing values in each column of the cleaned dataset:
 Company               0
Item                  0
Calories             15
Total Fat (g)        57
Saturated Fat (g)    57
Trans Fat (g)        57
Cholesterol (mg)      1
Sodium  (mg)          1
Carbs (g)            57
Fiber (g)            57
Sugars (g)           30
Protein (g)          57
dtype: int64


In [33]:
# Drop rows with missing values from the cleaned dataset
cleaned_df = cleaned_df.dropna()
print("\nFirst few rows of the cleaned dataset after dropping missing values:\n", cleaned_df.head())



First few rows of the cleaned dataset after dropping missing values:
       Company                           Item  Calories Total Fat (g)  \
0  McDonald’s                      Hamburger     250.0             9   
1  McDonald’s                   Cheeseburger     300.0            12   
2  McDonald’s            Double Cheeseburger     440.0            23   
3  McDonald’s                       McDouble     390.0            19   
4  McDonald’s   Quarter Pounder® with Cheese     510.0            26   

  Saturated Fat (g) Trans Fat (g) Cholesterol (mg) Sodium  (mg) Carbs (g)  \
0               3.5           0.5               25          520        31   
1                 6           0.5               40          750        33   
2                11           1.5               80         1150        34   
3                 8             1               65          920        33   
4                12           1.5               90         1190        40   

  Fiber (g)  Sugars (g) Protein (

In [34]:
missing_values_cleaned = cleaned_df.isnull().sum()
print("\nMissing values in each column of the cleaned dataset:\n", missing_values_cleaned)


Missing values in each column of the cleaned dataset:
 Company              0
Item                 0
Calories             0
Total Fat (g)        0
Saturated Fat (g)    0
Trans Fat (g)        0
Cholesterol (mg)     0
Sodium  (mg)         0
Carbs (g)            0
Fiber (g)            0
Sugars (g)           0
Protein (g)          0
dtype: int64


In [35]:
# Debug: Print the number of entries for each company
print(cleaned_df['Company'].value_counts())

Company
McDonald’s     328
KFC            218
Burger King    179
Wendy’s        154
Taco Bell      118
Pizza Hut       67
Name: count, dtype: int64


In [36]:
# Our working cleaned dataset is now ready for analysis under the use name of "cleaned_df"
cleaned_df.head()

,Company,Item,Calories,Total Fat (g),Saturated Fat (g),Trans Fat (g),Cholesterol (mg),Sodium (mg),Carbs (g),Fiber (g),Sugars (g),Protein (g)
0,McDonald’s,Hamburger,250.0,9,3.5,0.5,25,520,31,2,6.0,12
1,McDonald’s,Cheeseburger,300.0,12,6,0.5,40,750,33,2,6.0,15
2,McDonald’s,Double Cheeseburger,440.0,23,11,1.5,80,1150,34,2,7.0,25
3,McDonald’s,McDouble,390.0,19,8,1,65,920,33,2,7.0,22
4,McDonald’s,Quarter Pounder® with Cheese,510.0,26,12,1.5,90,1190,40,3,9.0,29


In [39]:
# import the data into a sqlite database
conn = sqlite3.connect('fastfood1.db')
cleaned_df.to_sql('table_columns', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()


In [41]:
# Connect to your database
conn = sqlite3.connect('fastfood1.db')

# Check existing tables
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)


            name
0  table_columns


In [42]:
# Connect to the SQLite database
conn = sqlite3.connect('fastfood1.db')

# Query to select all data from a specific table
query = "SELECT * FROM table_columns"

# Read the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Export the DataFrame to a CSV file
df.to_csv('cleaned_df.csv', index=False)

# Close the database connection
conn.close()